In [ ]:
%cd /content

# Clone the DVARF repository
!git clone https://github.com/AntoFratta/DVARF.git

%cd /content/DVARF

# Create a Colab-compatible requirements file by removing Windows-specific packages
!grep -v "triton-windows" requirements.txt > requirements_colab.txt

# Install all project dependencies
!pip install -r requirements_colab.txt

In [ ]:
%cd /content

# Clone the official SAM 3 repository from Meta
!git clone https://github.com/facebookresearch/sam3.git

In [ ]:
%cd /content/sam3

# Install SAM 3 in editable mode
!pip install -e .

In [ ]:
from huggingface_hub import login

# Login to Hugging Face (widget will prompt for your token)
login()

In [ ]:
%cd /content/DVARF

import sys
if "/content/DVARF" not in sys.path:
    sys.path.insert(0, "/content/DVARF")

from scripts.run_sam3_on_split import run_sam3_on_split

# Run SAM 3 on TRAIN split (generates predictions + semantic features)
run_sam3_on_split(
    split="train",
    score_threshold=0.26,
    max_images=None,
    save_segmentations=False,
    max_masks_per_image_per_class=None,
)

In [ ]:
%cd /content/DVARF

# Build the linear probe training dataset using semantic features
!python scripts/build_linear_probe_dataset.py

In [ ]:
%cd /content/DVARF

# Train the linear probe classifier on semantic features
!python scripts/train_linear_probe.py

In [ ]:
%cd /content/DVARF

import sys
if "/content/DVARF" not in sys.path:
    sys.path.insert(0, "/content/DVARF")

from scripts.run_sam3_on_split import run_sam3_on_split

# Run SAM 3 on TEST split (generates predictions + semantic features + segmentation masks)
run_sam3_on_split(
    split="test",
    score_threshold=0.26,
    max_images=None,
    save_segmentations=True,
    max_masks_per_image_per_class=None,
)

In [ ]:
%cd /content/DVARF

# Apply the trained linear probe to test predictions using semantic features
!python scripts/apply_linear_probe_to_split.py

In [ ]:
%cd /content/DVARF

# Evaluate the linear probe enhanced predictions and save metrics
!python scripts/eval_sam3_linear_probe_on_split.py | tee results/sam3_linear_probe_test_metrics.txt

In [ ]:
%cd /content/DVARF

# Archive zero-shot SAM 3 predictions on test set
!zip -r results/sam3_test_predictions.zip data/processed/predictions/sam3_yolo/test

# Archive linear probe enhanced predictions on test set
!zip -r results/sam3_linear_probe_test_predictions.zip data/processed/predictions/sam3_linear_probe_yolo/test

In [ ]:
%cd /content/DVARF

# Archive linear probe training artifacts (dataset and weights)
!zip -r results/sam3_linear_probe_training_artifacts.zip data/processed/linear_probe